In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim as gs
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')
import fasttext

In [2]:
true_pairs = pd.read_csv('../train_data/true_pairs_by_cosine_070_title_main_passage_keywords.csv', delimiter=',')
false_pairs = pd.read_csv('../train_data/false_pairs_by_pands_title_main_passage_keywords.csv', delimiter=',')
false_pairs = false_pairs.head((len(true_pairs))*1)

In [3]:
df_concat = pd.concat([true_pairs,false_pairs])

In [4]:
train_dataset, validation_dataset = train_test_split(df_concat, test_size=0.10, random_state=42)

In [5]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [6]:
MAX_PAD = 31

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [7]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        
        return embedding

In [8]:
train_article_title = []
train_table_title = []
train_label = []

for i,row in tqdm(train_dataset.iterrows()):
    
    train_article_title.append(create_embedding(row['article_page_title']))
    train_table_title.append(create_embedding(row['table_page_title']))
    train_label.append(row['label'])

169500it [00:52, 3218.83it/s]


In [9]:
train_article_title = np.array(train_article_title)
train_table_title = np.array(train_table_title)
train_label = np.array(train_label)

In [10]:
train_article_title.shape

(169500, 31, 50)

In [11]:
validation_article_title = []
validation_table_title = []
validation_label = []

for i, row in tqdm(validation_dataset.iterrows()):
    
    validation_article_title.append(create_embedding(row['article_page_title']))
    validation_table_title.append(create_embedding(row['table_page_title']))  
    validation_label.append(row['label'])

18834it [00:05, 3256.54it/s]


In [12]:
validation_article_title = np.array(validation_article_title)
validation_table_title = np.array(validation_table_title)
validation_label = np.array(validation_label)

In [13]:
validation_article_title.shape

(18834, 31, 50)

In [14]:
def affinity_method(query, document):
    
    transpose_document = tf.transpose(document,perm=[0, 2, 1])
    
    affinity = tf.matmul(query,transpose_document)
    
    normalize_affinity = tf.nn.softmax(affinity)
    
    return normalize_affinity

In [15]:
def attention_model():
    
    article_title = tf.keras.Input(shape=(31,50), dtype='float32')
    table_title = tf.keras.Input(shape=(31,50), dtype='float32')
    
    context_titleA = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_title)
    context_titleB = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_title)
    
    affinity_titleA_titleB = affinity_method(context_titleA,context_titleB)
    context_affinity = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(affinity_titleA_titleB)
    learned_vectorA = tf.keras.layers.Flatten()(context_affinity)
    
    affinity_embedding_titleA_titleB = affinity_method(article_title,table_title)
    context_affinity_embedding = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(affinity_embedding_titleA_titleB)
    learned_vectorB = tf.keras.layers.Flatten()(context_affinity_embedding)
    
    final_representation = tf.keras.layers.Concatenate()([learned_vectorA,learned_vectorB])
   
    MLP_input = tf.keras.layers.Dense(64, activation='relu')(final_representation)
    dropout1 = tf.keras.layers.Dropout(0.5, name="dropout1")(MLP_input)
    MLP_hidden = tf.keras.layers.Dense(32, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.5, name="dropout2")(MLP_hidden)
    MLP_output = tf.keras.layers.Dense(1,activation='sigmoid')(dropout2)
    
    model = tf.keras.Model(inputs=[article_title,table_title],outputs=MLP_output)
    
    return model

In [16]:
model = attention_model()

In [17]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=['accuracy'])

In [19]:
filepath="affinity_matrix_several_representations_model_title_{epoch:02d}_{val_accuracy:.4f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [20]:
history = model.fit([train_article_title, train_table_title], train_label, 
          epochs=10, 
          batch_size=32, 
          validation_data=([validation_article_title, validation_table_title], validation_label),
          callbacks=callbacks_list)

Train on 169500 samples, validate on 18834 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
169472/169500 [============================>.] - ETA: 0s - loss: 0.2340 - accuracy: 0.9106
Epoch 00001: val_accuracy improved from -inf to 0.93315, saving model to affinity_matrix_several_representations_model_title_01_0.9332.h5
169500/169500 [==============================] - 130s 770us/sample - loss: 0.2339 - accuracy: 0.9106 - val_loss: 0.1664 - val_accuracy: 0.9332
Epoch 2/10
169472/169500 [============================>.] - ETA: 0s - loss: 0.1713 - accuracy: 0.9357
Epoch 00002: val_accuracy did not improve from 0.93315
169500/169500 [==============================] - 121s 713us/sample - loss: 0.1713 - accuracy: 0.9358 - val_loss: 0.1740 - val_accuracy: 0.9281
Epoch 3/10
169472/169500 [============================>.] - ETA: 0s - loss: 0.1489 - accuracy: 0.9437
Epoch 00003: val_accuracy improved from 0.93315 to 0.94420, saving mod

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model = tf.keras.models.load_model('affinity_model_train_cossine.h5')

In [ ]:
y_prob = model.predict([validation_article_title,validation_table_title])
y_predict = (y_prob > 0.5).astype(np.int)
accuracy_score(validation_label, y_predict)

In [ ]:
import seaborn as sns
import sklearn.metrics
confusion_matrix = sklearn.metrics.confusion_matrix(validation_label, y_predict)

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="YlGnBu")#, annot_kws={"size": 15});
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predict label')
plt.show()